In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import statistics as st 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, plot_confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from math import sqrt

# EDA

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

### Remove listing_id  
#### listing_id is all unique without duplicates

In [3]:
train = train.drop(columns=['listing_id', 'property_details_url', 'elevation','property_name','address','title'])
test = test.drop(columns=['listing_id', 'property_details_url', 'elevation','property_name','address','title'])

## Adding Distances

### Add distance to commericial centres

In [4]:
ccdistance = pd.read_csv('auxiliary-data/sg-commerical-centres.csv')

In [5]:
# vectorized haversine function
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km


In [6]:
# train['cc_distance'] = haversine_np(train['lng'], train['lat'], ccdistance['lng'], ccdistance['lat'])
expected_result = pd.merge(train, ccdistance, on = 'planning_area')
expected_result.head()
train['cc_distance'] = haversine_np(train['lng'], train['lat'], expected_result['lng_y'], expected_result['lat_y'])

# train['cc_distance'] = haversine_np(train['lng'], train['lat'], ccdistance['lng'], ccdistance['lat'])
expected_result = pd.merge(test, ccdistance, on = 'planning_area')
expected_result.head()
test['cc_distance'] = haversine_np(test['lng'], test['lat'], expected_result['lng_y'], expected_result['lat_y'])

In [7]:
ccdistancemean = train['cc_distance'].mean()
train['cc_distance']=train['cc_distance'].fillna(ccdistancemean)

ccdistancemean = test['cc_distance'].mean()
test['cc_distance']=test['cc_distance'].fillna(ccdistancemean)

### Add distance to mrt stations

In [8]:
# using long and lat
mrtdistance = pd.read_csv('auxiliary-data/sg-mrt-stations.csv')
expected_result = pd.merge(train, mrtdistance, on = 'planning_area')
train['mrt_distance'] = haversine_np(train['lng'], train['lat'], expected_result['lng_y'], expected_result['lat_y'])


# using long and lat
mrtdistance = pd.read_csv('auxiliary-data/sg-mrt-stations.csv')
expected_result = pd.merge(test, mrtdistance, on = 'planning_area')
test['mrt_distance'] = haversine_np(test['lng'], test['lat'], expected_result['lng_y'], expected_result['lat_y'])

### Add distance to primary school 

In [9]:
# using long and lat
primaryschooldistance = pd.read_csv('auxiliary-data/sg-primary-schools.csv')
expected_result = pd.merge(train, primaryschooldistance, on = 'planning_area')
train['primary_school_distance'] = haversine_np(train['lng'], train['lat'], expected_result['lng_y'], expected_result['lat_y'])

# using long and lat
primaryschooldistance = pd.read_csv('auxiliary-data/sg-primary-schools.csv')
expected_result = pd.merge(test, primaryschooldistance, on = 'planning_area')
test['primary_school_distance'] = haversine_np(test['lng'], test['lat'], expected_result['lng_y'], expected_result['lat_y'])

### Add distance to secondary school

In [10]:
# using long and lat
secschooldistance = pd.read_csv('auxiliary-data/sg-secondary-schools.csv')
expected_result = pd.merge(train, secschooldistance, on = 'planning_area')
train['secondary_school_distance'] = haversine_np(train['lng'], train['lat'], expected_result['lng_y'], expected_result['lat_y'])

# using long and lat
secschooldistance = pd.read_csv('auxiliary-data/sg-secondary-schools.csv')
expected_result = pd.merge(test, secschooldistance, on = 'planning_area')
test['secondary_school_distance'] = haversine_np(test['lng'], test['lat'], expected_result['lng_y'], expected_result['lat_y'])

### Add distance to shopping malls

In [11]:
# using long and lat
shop_distance = pd.read_csv('auxiliary-data/sg-shopping-malls.csv')
expected_result = pd.merge(train, secschooldistance, on = 'planning_area')
train['shop_distance'] = haversine_np(train['lng'], train['lat'], expected_result['lng_y'], expected_result['lat_y'])

# using long and lat
shop_distance = pd.read_csv('auxiliary-data/sg-shopping-malls.csv')
expected_result = pd.merge(test, secschooldistance, on = 'planning_area')
test['shop_distance'] = haversine_np(test['lng'], test['lat'], expected_result['lng_y'], expected_result['lat_y'])

## Property Type cleaning

In [12]:
train.property_type.unique()

array(['hdb 4 rooms', 'hdb', 'condo', 'Condo', 'bungalow', 'Hdb',
       'terraced house', 'Hdb Executive', 'apartment',
       'Semi-Detached House', 'Apartment', 'Hdb 4 Rooms',
       'semi-detached house', 'hdb 3 rooms', 'executive condo',
       'corner terrace', 'hdb executive', 'Hdb 3 Rooms', 'Hdb 5 Rooms',
       'hdb 5 rooms', 'landed', 'hdb 2 rooms', 'Executive Condo',
       'Bungalow', 'Corner Terrace', 'Terraced House', 'cluster house',
       'Cluster House', 'Land Only', 'townhouse', 'Hdb 2 Rooms',
       'conservation house', 'land only', 'walk-up', 'Townhouse',
       'Conservation House', 'good class bungalow', 'Landed', 'shophouse'],
      dtype=object)

In [13]:
#Change all to uppercase
train['property_type']=train['property_type'].str.upper()

#Change all to uppercase
test['property_type']=test['property_type'].str.upper()

In [14]:
train.property_type.unique()

array(['HDB 4 ROOMS', 'HDB', 'CONDO', 'BUNGALOW', 'TERRACED HOUSE',
       'HDB EXECUTIVE', 'APARTMENT', 'SEMI-DETACHED HOUSE', 'HDB 3 ROOMS',
       'EXECUTIVE CONDO', 'CORNER TERRACE', 'HDB 5 ROOMS', 'LANDED',
       'HDB 2 ROOMS', 'CLUSTER HOUSE', 'LAND ONLY', 'TOWNHOUSE',
       'CONSERVATION HOUSE', 'WALK-UP', 'GOOD CLASS BUNGALOW',
       'SHOPHOUSE'], dtype=object)

In [15]:
# TO DO: MAY NEED TO CHECK EACH VALUE INDIVIDUALLY TO SEE IF THERE ARE ANY WITH ONLY 1 

In [16]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train['property_type'])
train['property_type']=le.transform(train['property_type'])

test['property_type']=le.transform(test['property_type'])

## Tenure 

In [17]:
train.tenure.unique()

array([nan, '99-year leasehold', 'freehold', '999-year leasehold',
       '110-year leasehold', '946-year leasehold', '103-year leasehold',
       '956-year leasehold', '929-year leasehold', '102-year leasehold',
       '100-year leasehold', '947-year leasehold'], dtype=object)

In [18]:
train['tenure'] = train['tenure'].replace(
    [
    '99-year leasehold',
    '110-year leasehold',
    '103-year leasehold',
    '102-year leasehold',
    '100-year leasehold',
    ],
    '99-year leasehold'
)

train['tenure'] = train['tenure'].replace(
    [
    '946-year leasehold',
    '999-year leasehold',
    '956-year leasehold',
    '929-year leasehold',
    '947-year leasehold',
    ],
    '999-year leasehold'
)



test['tenure'] = test['tenure'].replace(
    [
    '99-year leasehold',
    '110-year leasehold',
    '103-year leasehold',
    '102-year leasehold',
    '100-year leasehold',
    ],
    '99-year leasehold'
)

test['tenure'] = test['tenure'].replace(
    [
    '946-year leasehold',
    '999-year leasehold',
    '956-year leasehold',
    '929-year leasehold',
    '947-year leasehold',
    ],
    '999-year leasehold'
)

In [19]:
train.tenure.unique()

array([nan, '99-year leasehold', 'freehold', '999-year leasehold'],
      dtype=object)

In [20]:
test['tenure'] = test['tenure'].fillna('99-year leasehold')

In [21]:
## Remove NA 
train = train.dropna(subset=['tenure'])

## Remove NA 
test['tenure'] = test['tenure'].fillna('99-year leasehold')

In [22]:
from sklearn import preprocessing
tenurepp = preprocessing.LabelEncoder()
tenurepp.fit(train['tenure'])
train['tenure']=tenurepp.transform(train['tenure'])

test['tenure']=tenurepp.transform(test['tenure'])

### Funishing

In [23]:
train.furnishing.unique()

array(['unspecified', 'partial', 'unfurnished', 'fully', 'na'],
      dtype=object)

In [24]:
#Assume not specified and na as unfurnished 
train['furnishing'] = train['furnishing'].replace(
    [
    'na',
    'unspecified',
    ],
    'unfurnished'
)

#Assume not specified and na as unfurnished 
test['furnishing'] = test['furnishing'].replace(
    [
    'na',
    'unspecified',
    ],
    'unfurnished'
)

In [25]:
train['furnishing'].unique()

array(['unfurnished', 'partial', 'fully'], dtype=object)

In [26]:
from sklearn import preprocessing
furnishing = preprocessing.LabelEncoder()
furnishing.fit(train['furnishing'])
train['furnishing']=furnishing.transform(train['furnishing'])

test['furnishing']=furnishing.transform(test['furnishing'])

## floor_level 

In [27]:
# Too few data for floor level, we should just take out the column
train = train.drop(columns='floor_level')

# Too few data for floor level, we should just take out the column
test = test.drop(columns='floor_level')

## total_num_units

In [28]:
meantotalunit = train['total_num_units'].mean()
train['total_num_units'].fillna(meantotalunit, inplace=True)

meantotalunit = test['total_num_units'].mean()
test['total_num_units'].fillna(meantotalunit, inplace=True)

## build year

In [29]:
avgbuildyear = round(train['built_year'].mean(),0)
train['built_year'] = train['built_year'].fillna(avgbuildyear)

avgbuildyear = round(test['built_year'].mean(),0)
test['built_year'] = test['built_year'].fillna(avgbuildyear)

## num_baths

In [30]:
avgbath = round(train['num_baths'].mean(),1)
train['num_baths'] = train['num_baths'].fillna(avgbath)

avgbath = round(test['num_baths'].mean(),1)
test['num_baths'] = test['num_baths'].fillna(avgbath)

## num_bedds

In [31]:
avgbed = round(train['num_beds'].mean(),1)
train['num_beds'] = train['num_beds'].fillna(avgbath)

avgbed = round(test['num_beds'].mean(),1)
test['num_beds'] = test['num_beds'].fillna(avgbath)

## subzone&planning zone are very important to price of realestate. So we can drop na listings without it.

## num_avaliable_units

In [32]:
numavaunittypes = train.available_unit_types.str.count(',') +1
train['available_unit_types'] = numavaunittypes

In [33]:
avg_ava_units = train['available_unit_types'].mean()
train['available_unit_types'] = train['available_unit_types'].fillna(avg_ava_units)

In [34]:
numavaunittypes = test.available_unit_types.str.count(',') +1
test['available_unit_types'] = numavaunittypes

avg_ava_units = test['available_unit_types'].mean()
test['available_unit_types'] = test['available_unit_types'].fillna(avg_ava_units)

## Subzone & planning zone

In [35]:
test.count()

property_type                7000
tenure                       7000
built_year                   7000
num_beds                     7000
num_baths                    7000
size_sqft                    7000
furnishing                   7000
available_unit_types         7000
total_num_units              7000
lat                          7000
lng                          7000
subzone                      6967
planning_area                6967
cc_distance                  7000
mrt_distance                 7000
primary_school_distance      7000
secondary_school_distance    7000
shop_distance                7000
dtype: int64

In [36]:
train['planning_area'].unique()

array(['serangoon', 'marine parade', 'bukit timah', 'downtown core',
       'bukit batok', 'kallang', 'tanglin', 'bedok', 'bishan', 'novena',
       'ang mo kio', 'queenstown', 'pasir ris', 'museum', 'newton', nan,
       'yishun', 'orchard', 'bukit panjang', 'sembawang', 'geylang',
       'bukit merah', 'sengkang', 'toa payoh', 'clementi', 'outram',
       'woodlands', 'hougang', 'singapore river', 'punggol',
       'river valley', 'jurong west', 'southern islands', 'tampines',
       'rochor', 'jurong east', 'choa chu kang',
       'central water catchment', 'tengah', 'lim chu kang', 'changi',
       'seletar', 'paya lebar', 'mandai'], dtype=object)

In [37]:
pa = train['planning_area'] + test['planning_area']
pa = pa.dropna().drop_duplicates().unique()
pa = pd.DataFrame(pa)
pa.count()

0    1255
dtype: int64

In [38]:
a = train['planning_area'].drop_duplicates()
b = test['planning_area'].drop_duplicates()
c = pd.concat([a, b]).dropna()
c = c.drop_duplicates()


In [39]:
from sklearn import preprocessing
planningzone = preprocessing.LabelEncoder()
planningzone.fit(c)

train = train.dropna()
train['planning_area']=planningzone.transform(train['planning_area'])

In [53]:
mostfreq = test['planning_area'].mode()
mostfreq
test['planning_area'] = test['planning_area'].fillna('bukit timah')
test['planning_area']=planningzone.transform(test['planning_area'])

## CAN CONSIDER NOT DROPPING SUBZONE

In [56]:
train = train.drop(columns='subzone')
test = test.drop(columns='subzone')

## Data MINING

In [77]:
'''Main'''
import numpy as np
import pandas as pd
from time import time

'''Data Prep'''
from sklearn import preprocessing as pp
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report

# data visualisation
import matplotlib.pyplot as plt

from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier as xgb

In [62]:
dataX = train.copy()
dataY = dataX.pop('price')

X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size=0.2, random_state=88)

In [104]:
# important sample code
# parameter tuning for random forest
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

params = {'max_features':('sqrt', 'log2'), 'n_estimators':range(10, 100, 5)} # from 10 to 100 with 5 increment

clf = RandomForestClassifier(
    n_jobs=-1, # for using multiple cpu/core to speed up calculation
    random_state=2022, # random seed
    verbose=0, # print out details or not    
)

clf2 = GridSearchCV(clf, param_grid=params, scoring='accuracy', cv=5)
clf2.fit(X_train, y_train)

# see all output variables in cv results
sorted(clf2.cv_results_.keys())

In [ ]:
clf2.best_score_

In [ ]:
# get the best model
clf2.best_estimator_

In [ ]:
print("Accuracy on training set: {:.3f}".format(clf2.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(clf2.score(X_test, y_test)))